## MinMaxScaler 전체 코드

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import ndcg_score
from sentence_transformers import SentenceTransformer
import warnings

# 경고 메시지 무시 (깔끔한 결과 확인용)
warnings.filterwarnings('ignore')

# 1. 데이터 로드 및 타겟(Relevance) 설정
df = pd.read_csv('C:\\python\\team_project\\sports-analysis-fighter\\reranked_final_data4.csv')

# [핵심] 랭킹 모델용 점수 변환: 순위가 높을수록(1위) 높은 점수를 부여
df['relevance'] = df['llm_rank'].max() - df['llm_rank'] + 1

# 2. 질문 벡터화 및 차원 축소 (PCA)
print("Step 2: 질문 벡터화 및 PCA 진행 중...")
model_nlp = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
unique_questions = df['질문'].unique()
q_embeddings = model_nlp.encode(unique_questions)

# 768차원 -> 5차원으로 축소하여 모델 피처로 사용
pca = PCA(n_components=5, random_state=42)
q_pca = pca.fit_transform(q_embeddings)
q_pca_dict = {q: v for q, v in zip(unique_questions, q_pca)}

pca_cols = [f'pca_{i}' for i in range(5)]
df[pca_cols] = pd.DataFrame(df['질문'].map(q_pca_dict).tolist(), index=df.index)

# 3. 질문 클러스터링 (K-Means) - 범주형 피처 생성
print("Step 3: 질문 클러스터링 진행 중...")
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
q_clusters = kmeans.fit_predict(q_embeddings)
q_cluster_dict = {q: c for q, c in zip(unique_questions, q_clusters)}
df['q_cluster'] = df['질문'].map(q_cluster_dict)

# ---------------------------------------------------------
# 4. 데이터 성격별 분리 전처리 (리더 조언 반영)
# ---------------------------------------------------------
print("Step 4: 데이터 성격별 분리 전처리 시작...")

# A. 수치형 피처: MinMaxScaler 적용 (0~1 사이로 정규화)
score_cols = ['sbert_score', 'n2v_score', 'vector_score', '매칭 스코어']
scaler = MinMaxScaler()
df[score_cols] = scaler.fit_transform(df[score_cols])

# B. 범주형 피처: LabelEncoder 적용 (정수형 유지, 스케일링 제외)
le_league = LabelEncoder()
df['league_encoded'] = le_league.fit_transform(df['추천 리그'])
# q_cluster는 이미 정수형이므로 그대로 사용

# 5. 데이터 분할 (질문 그룹 기준)
features = score_cols + ['league_encoded', 'q_cluster'] + pca_cols
X = df[features]
y = df['relevance']
groups = df['질문']

# [중요] 질문이 섞이지 않도록 8:2 분할
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups))

# 정렬을 통해 그룹 정보 일치시키기
train_data = df.iloc[train_idx].sort_values('질문')
test_data = df.iloc[test_idx].sort_values('질문')

X_train = train_data[features]
y_train = train_data['relevance']
group_train = train_data.groupby('질문', sort=False).size().values

X_test = test_data[features]
y_test = test_data['relevance']
group_test = test_data.groupby('질문', sort=False).size().values

# 6. 모델 학습 (LGBMRanker)
print("Step 6: LGBMRanker 학습 시작...")
# 범주형 변수 리스트 지정 (모델이 내부 최적화 수행)
categorical_features = ['league_encoded', 'q_cluster']

model = lgb.LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    learning_rate=0.05,
    max_depth=5,
    n_estimators=200,
    random_state=42,
    importance_type='gain'
)

model.fit(
    X_train, y_train,
    group=group_train,
    eval_set=[(X_test, y_test)],
    eval_group=[group_test],
    eval_at=[1, 3, 5],
    categorical_feature=categorical_features  # 범주형 피처 명시
)

# 7. 성능 평가 (Mean NDCG)
preds = model.predict(X_test)
test_data['preds'] = preds

ndcg_list = []
for q in test_data['질문'].unique():
    q_subset = test_data[test_data['질문'] == q]
    if len(q_subset) > 1:
        # 질문별 NDCG 계산 후 리스트에 추가
        score = ndcg_score([q_subset['relevance']], [q_subset['preds']])
        ndcg_list.append(score)

print("\n" + "="*30)
print(f"✅ 최종 결과 Mean NDCG: {np.mean(ndcg_list):.4f}")
print("="*30)

# 8. 피처 중요도 확인 (발표용)
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
feat_imp = pd.Series(model.feature_importances_, index=features).sort_values(ascending=False)
sns.barplot(x=feat_imp, y=feat_imp.index)
plt.title('Feature Importance (LGBMRanker)')
plt.show()

c:\Users\ghks0\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Step 2: 질문 벡터화 및 PCA 진행 중...
Step 3: 질문 클러스터링 진행 중...
Step 4: 데이터 성격별 분리 전처리 시작...
Step 6: LGBMRanker 학습 시작...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000320 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1762
[LightGBM] [Info] Number of data points in the train set: 14260, number of used features: 11
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g